In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM,GRU, Dense, Bidirectional
from tensorflow.keras.layers import SimpleRNN, Dense
from tensorflow.keras.layers import TimeDistributed, Flatten, Dropout
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from sklearn.preprocessing import MinMaxScaler

SEQ_LENGTH = 24  # for 24 hours sequence

model = Sequential([
    LSTM(50, return_sequences=True, input_shape=(24, 6)),
    Dropout(0.2),
    LSTM(50),
    Dropout(0.2),
    Dense(1)
])


In [2]:
model = tf.keras.models.load_model ("../Checkpoints/A_A_LSTMmodel.h5")

In [110]:
df = pd.read_csv("../Data/Predict.csv")

In [111]:
seasons_mapping = {
    'Spring': 0,
    'Summer': 1,
    'Autumn': 2,
    'Winter': 3
}

df['season'] = df['season'].map(seasons_mapping)

In [74]:
df

,season,year,month,day_of_month,hour,Consumption_MWh
0,1,2024,1,15,1,145.018660
1,1,2024,1,15,2,153.871964
2,1,2024,1,15,3,161.731316
3,1,2024,1,15,4,168.106920
4,1,2024,1,15,5,173.246406
5,1,2024,1,15,6,178.585902
6,1,2024,1,15,7,175.353635
7,1,2024,1,15,8,169.700589
8,1,2024,1,15,9,164.440383
9,1,2024,1,15,10,165.027396


In [112]:
scaler = MinMaxScaler()
scaled_df = scaler.fit_transform(df[['season', 'year', 'month', 'day_of_month', 'hour', 'Consumption_MWh']])

scaled_df

array([[0.        , 0.        , 0.        , 0.        , 0.        ,
        0.22092037],
       [0.        , 0.        , 0.        , 0.        , 0.04347826,
        0.40673152],
       [0.        , 0.        , 0.        , 0.        , 0.08695652,
        0.57168184],
       [0.        , 0.        , 0.        , 0.        , 0.13043478,
        0.70549158],
       [0.        , 0.        , 0.        , 0.        , 0.17391304,
        0.81335796],
       [0.        , 0.        , 0.        , 0.        , 0.2173913 ,
        0.9254221 ],
       [0.        , 0.        , 0.        , 0.        , 0.26086957,
        0.85758401],
       [0.        , 0.        , 0.        , 0.        , 0.30434783,
        0.73893915],
       [0.        , 0.        , 0.        , 0.        , 0.34782609,
        0.62853913],
       [0.        , 0.        , 0.        , 0.        , 0.39130435,
        0.64085922],
       [0.        , 0.        , 0.        , 0.        , 0.43478261,
        0.73887946],
       [0.        , 0

In [113]:
predict = np.array([scaled_df])

In [82]:
predict

array([[[0.        , 0.        , 0.        , 0.        , 0.        ,
         0.22092037],
        [0.        , 0.        , 0.        , 0.        , 0.04347826,
         0.40673152],
        [0.        , 0.        , 0.        , 0.        , 0.08695652,
         0.57168184],
        [0.        , 0.        , 0.        , 0.        , 0.13043478,
         0.70549158],
        [0.        , 0.        , 0.        , 0.        , 0.17391304,
         0.81335796],
        [0.        , 0.        , 0.        , 0.        , 0.2173913 ,
         0.9254221 ],
        [0.        , 0.        , 0.        , 0.        , 0.26086957,
         0.85758401],
        [0.        , 0.        , 0.        , 0.        , 0.30434783,
         0.73893915],
        [0.        , 0.        , 0.        , 0.        , 0.34782609,
         0.62853913],
        [0.        , 0.        , 0.        , 0.        , 0.39130435,
         0.64085922],
        [0.        , 0.        , 0.        , 0.        , 0.43478261,
         0.73887946],

In [114]:
predict_result = model.predict (predict)

1/1 [==============================] - 0s 168ms/step


In [ ]:

# Extract the last sequence of data for prediction
input_sequence = scaled_df[-SEQ_LENGTH:]

# Initialize with input sequence
full_sequence = np.copy(input_sequence)

# The last column is assumed to be 'Consumption_MWh', replace it with the prediction result
full_sequence[:, -1] = predict_result

# Inverse transform the entire sequence
inverse_transformed_sequence = scaler.inverse_transform(full_sequence)

# Extract the 'Consumption_MWh' part
predicted_consumption = inverse_transformed_sequence[:, -1]

print(predicted_consumption)

In [121]:
from datetime import datetime, timedelta

def add_one_hour(season, year, month, day_of_month, hour):
    # Construct a datetime object from the provided values
    dt = datetime(year, month, day_of_month, hour)
    
    # Add one hour
    dt += timedelta(hours=1)
    
    # Extract the new values
    new_year = dt.year
    new_month = dt.month
    new_day_of_month = dt.day
    new_hour = dt.hour
    
    if new_month in [1, 2, 3]:
        new_season = 0
    elif new_month in [4, 5, 6]:
        new_season = 1
    elif new_month in [7, 8, 9]:
        new_season = 2
    else:
        new_season = 3
        
    return new_season, new_year, new_month, new_day_of_month, new_hour

In [124]:
def denormalized_value (value):
    year = df['year'].max()
    hour = df['hour'].max()
    month = df['month'].max()
    day_of_month = df['day_of_month'].max()
    seasons = df['season'].max()
     
    season, year, month, day_of_month, hour = add_one_hour(seasons, year, month, day_of_month, hour)
    return scaler.inverse_transform([[season, year, month, day_of_month, hour, value]])[0][5]

In [125]:
denormalized_value (result[0][0])

146.6041051027298